In [21]:
from src.constants import fn_osm_db_dc

import src.osm.osmdb_constants; reload(src.osm.osmdb_constants)
from src.osm.osmdb_constants import *

import src.osm.osmdb_filter; reload(src.osm.osmdb_filter)
from src.osm.osmdb_filter import *

In [3]:
osm_db_dc = 'data/'+fn_osm_db_dc
fn_osm_db_dc, osm_db_dc

('osm_db_dc.sqlite3', 'data/osm_db_dc.sqlite3')

# func get_geom_by_otoid

In [3]:
geom = get_geom_by_otoid(osm_db_dc, 'N',30066952)
print geom 
geom = get_geom_by_otoid(osm_db_dc, 'W',30066952)

POINT (-76.9926346 38.8812125)
no geometry match ot=W, oid=30066952 in db: data/osm_db_dc.sqlite3


# func filter_geom_by_otoids

In [4]:
otoids = [('N', 30066952)]
filter_geom_by_otoids(osm_db_dc, otoids, debug=True)

otoids = [('N', 30066952),('N', 49716326),('W', 97167308)]
rows = filter_geom_by_otoids(osm_db_dc, otoids, debug=True)

otoids = [('N', 30066952),('N', 49716326),('W', 97167308), ('R',162069)]
gpdf = filter_geom_by_otoids_to_gpdf(osm_db_dc, otoids)
print type(gpdf.iloc[3].geometry)
gpdf.iloc[3:,:]  # geocollection can't be plot nor intersect

SELECT * FROM geometry NATURAL JOIN (SELECT 'N' AS ot, 30066952 AS oid)
SELECT * FROM geometry NATURAL JOIN (SELECT 'N' AS ot, 30066952 AS oid UNION ALL SELECT 'N', 49716326 UNION ALL SELECT 'W', 97167308)
<class 'shapely.geometry.collection.GeometryCollection'>


,ot,oid,geometry
3,R,162069,"(POINT (-77.0366456 38.8949549), POLYGON ((-77..."


# Test cases for filter_tbtag

In [5]:
filter_tbtag(osm_db_dc, debug=True)  # no where clause
filter_tbtag(osm_db_dc, ot='N', debug=True)  # only where ot=ot
filter_tbtag(osm_db_dc, in_tags=(('highway', None ),), debug=True)  # only in_tags, has key
filter_tbtag(osm_db_dc, in_tags=(('highway', None ),), ot='N', debug=True)  # ot + in_tags(has key)
filter_tbtag(osm_db_dc, in_tags=(('highway', 'stop' ),), ot='N', debug=True)  # ot + in_tags(some_k=some_v)

filter table tag SELECT * FROM tag
# rows 1484405
ot: [u'N' u'W' u'R']
# keys 919

filter table tag SELECT * FROM tag WHERE (ot='N')
# rows 157884
ot: [u'N']
# keys 531

filter table tag SELECT * FROM tag WHERE ((key='highway'))
# rows 23232
ot: [u'N' u'W' u'R']
# keys 1
keys: [u'highway']
key = highway value: [u'stop' u'crossing' u'traffic_signals' u'give_way' u'motorway_junction'
 u'turning_circle' u'mini_roundabout' u'turning_loop' u'speed_camera'
 u'bus_stop' u'emergency_access_point' u'elevator' u'milestone'
 u'residential' u'footway' u'secondary' u'primary_link' u'tertiary'
 u'primary' u'trunk' u'unclassified' u'motorway' u'trunk_link'
 u'motorway_link' u'service' u'secondary_link' u'tertiary_link' u'path'
 u'pedestrian' u'cycleway' u'steps' u'construction' u'track' u'services'
 u'yes' u'living_street' u'bridleway']

filter table tag SELECT * FROM tag WHERE (ot='N') AND ((key='highway'))
# rows 4703
ot: [u'N']
# keys 1
keys: [u'highway']
key = highway value: [u'stop' u'crossing' 

[(u'N', 49715946, u'highway', u'stop'),
 (u'N', 49715951, u'highway', u'stop'),
 (u'N', 49715974, u'highway', u'stop'),
 (u'N', 49716879, u'highway', u'stop'),
 (u'N', 49716928, u'highway', u'stop'),
 (u'N', 49716962, u'highway', u'stop'),
 (u'N', 49723161, u'highway', u'stop'),
 (u'N', 49724802, u'highway', u'stop'),
 (u'N', 49724849, u'highway', u'stop'),
 (u'N', 49725736, u'highway', u'stop'),
 (u'N', 49726188, u'highway', u'stop'),
 (u'N', 49732057, u'highway', u'stop'),
 (u'N', 49732072, u'highway', u'stop'),
 (u'N', 49734010, u'highway', u'stop'),
 (u'N', 49734413, u'highway', u'stop'),
 (u'N', 49734802, u'highway', u'stop'),
 (u'N', 49734873, u'highway', u'stop'),
 (u'N', 49737931, u'highway', u'stop'),
 (u'N', 49739319, u'highway', u'stop'),
 (u'N', 49739803, u'highway', u'stop'),
 (u'N', 49739997, u'highway', u'stop'),
 (u'N', 49740100, u'highway', u'stop'),
 (u'N', 49740442, u'highway', u'stop'),
 (u'N', 49740477, u'highway', u'stop'),
 (u'N', 49740585, u'highway', u'stop'),


In [6]:
filter_tbtag(osm_db_dc, in_tags=(('highway', ('stop','traffic_signals','crossing')),), ot='N', debug=True)  # ot + in_tags(multiple values of some_k)
filter_tbtag(osm_db_dc, in_tags=(('traffic_signals', 'signal'),), ot='N', debug=True)  # for comparision of whether multiple keys works as expected
filter_tbtag(osm_db_dc, in_tags=(('highway', ('stop','traffic_signals','crossing')),('traffic_signals', 'signal')), ot='N', debug=True)  # ot + in_tags(multiple keys, multiple values for one key)

filter table tag SELECT * FROM tag WHERE (ot='N') AND ((key='highway' and (value='stop' or value='traffic_signals' or value='crossing')))
# rows 4217
ot: [u'N']
# keys 1
keys: [u'highway']
key = highway value: [u'stop' u'crossing' u'traffic_signals']

filter table tag SELECT * FROM tag WHERE (ot='N') AND ((key='traffic_signals' and value='signal'))
# rows 176
ot: [u'N']
# keys 1
keys: [u'traffic_signals']
key = traffic_signals value: [u'signal']

filter table tag SELECT * FROM tag WHERE (ot='N') AND ((key='highway' and (value='stop' or value='traffic_signals' or value='crossing')) or (key='traffic_signals' and value='signal'))
# rows 4393
ot: [u'N']
# keys 2
keys: [u'highway' u'traffic_signals']
key = highway value: [u'stop' u'crossing' u'traffic_signals']
key = traffic_signals value: [u'signal']



[(u'N', 49715946, u'highway', u'stop'),
 (u'N', 49715951, u'highway', u'stop'),
 (u'N', 49715974, u'highway', u'stop'),
 (u'N', 49716326, u'highway', u'crossing'),
 (u'N', 49716545, u'highway', u'traffic_signals'),
 (u'N', 49716545, u'traffic_signals', u'signal'),
 (u'N', 49716602, u'highway', u'traffic_signals'),
 (u'N', 49716602, u'traffic_signals', u'signal'),
 (u'N', 49716610, u'highway', u'traffic_signals'),
 (u'N', 49716610, u'traffic_signals', u'signal'),
 (u'N', 49716662, u'highway', u'traffic_signals'),
 (u'N', 49716664, u'highway', u'crossing'),
 (u'N', 49716680, u'highway', u'traffic_signals'),
 (u'N', 49716879, u'highway', u'stop'),
 (u'N', 49716928, u'highway', u'stop'),
 (u'N', 49716962, u'highway', u'stop'),
 (u'N', 49717245, u'highway', u'traffic_signals'),
 (u'N', 49717477, u'highway', u'traffic_signals'),
 (u'N', 49717954, u'highway', u'traffic_signals'),
 (u'N', 49718053, u'highway', u'crossing'),
 (u'N', 49718775, u'highway', u'traffic_signals'),
 (u'N', 49718775, u

In [7]:
filter_tbtag(osm_db_dc, in_tags=(('highway', ('stop','traffic_signals','crossing')),('traffic_signals', 'signal')), ot='N', debug=True)  # ot + in_tags(multiple keys, multiple values for one key)
filter_tbtag(osm_db_dc, debug=True, in_tags=(('highway', ('stop','traffic_signals','crossing')),('traffic_signals', 'signal')), ex_tags=(('highway','stop'),), ot='N')  # ot + in_tags + ex_tags(some_k=some_v)
filter_tbtag(osm_db_dc, debug=True, in_tags=(('highway','stop'),), ot='N')  # for comparision of whether ex_tags(some_k=some_v) works as expected


filter table tag SELECT * FROM tag WHERE (ot='N') AND ((key='highway' and (value='stop' or value='traffic_signals' or value='crossing')) or (key='traffic_signals' and value='signal'))
# rows 4393
ot: [u'N']
# keys 2
keys: [u'highway' u'traffic_signals']
key = highway value: [u'stop' u'crossing' u'traffic_signals']
key = traffic_signals value: [u'signal']

filter table tag SELECT * FROM tag WHERE (ot='N') AND ((key='highway' and (value='stop' or value='traffic_signals' or value='crossing')) or (key='traffic_signals' and value='signal')) AND (not((key='highway' and value='stop')))
# rows 3989
ot: [u'N']
# keys 2
keys: [u'highway' u'traffic_signals']
key = highway value: [u'crossing' u'traffic_signals']
key = traffic_signals value: [u'signal']

filter table tag SELECT * FROM tag WHERE (ot='N') AND ((key='highway' and value='stop'))
# rows 404
ot: [u'N']
# keys 1
keys: [u'highway']
key = highway value: [u'stop']



[(u'N', 49715946, u'highway', u'stop'),
 (u'N', 49715951, u'highway', u'stop'),
 (u'N', 49715974, u'highway', u'stop'),
 (u'N', 49716879, u'highway', u'stop'),
 (u'N', 49716928, u'highway', u'stop'),
 (u'N', 49716962, u'highway', u'stop'),
 (u'N', 49723161, u'highway', u'stop'),
 (u'N', 49724802, u'highway', u'stop'),
 (u'N', 49724849, u'highway', u'stop'),
 (u'N', 49725736, u'highway', u'stop'),
 (u'N', 49726188, u'highway', u'stop'),
 (u'N', 49732057, u'highway', u'stop'),
 (u'N', 49732072, u'highway', u'stop'),
 (u'N', 49734010, u'highway', u'stop'),
 (u'N', 49734413, u'highway', u'stop'),
 (u'N', 49734802, u'highway', u'stop'),
 (u'N', 49734873, u'highway', u'stop'),
 (u'N', 49737931, u'highway', u'stop'),
 (u'N', 49739319, u'highway', u'stop'),
 (u'N', 49739803, u'highway', u'stop'),
 (u'N', 49739997, u'highway', u'stop'),
 (u'N', 49740100, u'highway', u'stop'),
 (u'N', 49740442, u'highway', u'stop'),
 (u'N', 49740477, u'highway', u'stop'),
 (u'N', 49740585, u'highway', u'stop'),


In [8]:
filter_tbtag(osm_db_dc, in_tags=(('highway', ('stop','traffic_signals','crossing')),('traffic_signals', 'signal')), ot='N', debug=True)  # ot + in_tags(multiple keys, multiple values for one key)
filter_tbtag(osm_db_dc, debug=True, in_tags=(('highway', ('stop','traffic_signals','crossing')),('traffic_signals', 'signal')), ex_tags=(('highway',['stop','crossing']),), ot='N')  # ot + in_tags + ex_tags(multiple values of some_k)
filter_tbtag(osm_db_dc, debug=True, in_tags=(('highway',['stop','crossing']),), ot='N')  # for comparision of whether ex_tags(multiple values of some_k) works as expected


filter table tag SELECT * FROM tag WHERE (ot='N') AND ((key='highway' and (value='stop' or value='traffic_signals' or value='crossing')) or (key='traffic_signals' and value='signal'))
# rows 4393
ot: [u'N']
# keys 2
keys: [u'highway' u'traffic_signals']
key = highway value: [u'stop' u'crossing' u'traffic_signals']
key = traffic_signals value: [u'signal']

filter table tag SELECT * FROM tag WHERE (ot='N') AND ((key='highway' and (value='stop' or value='traffic_signals' or value='crossing')) or (key='traffic_signals' and value='signal')) AND (not((key='highway' and (value='stop' or value='crossing'))))
# rows 1630
ot: [u'N']
# keys 2
keys: [u'highway' u'traffic_signals']
key = highway value: [u'traffic_signals']
key = traffic_signals value: [u'signal']

filter table tag SELECT * FROM tag WHERE (ot='N') AND ((key='highway' and (value='stop' or value='crossing')))
# rows 2763
ot: [u'N']
# keys 1
keys: [u'highway']
key = highway value: [u'stop' u'crossing']



[(u'N', 49715946, u'highway', u'stop'),
 (u'N', 49715951, u'highway', u'stop'),
 (u'N', 49715974, u'highway', u'stop'),
 (u'N', 49716326, u'highway', u'crossing'),
 (u'N', 49716664, u'highway', u'crossing'),
 (u'N', 49716879, u'highway', u'stop'),
 (u'N', 49716928, u'highway', u'stop'),
 (u'N', 49716962, u'highway', u'stop'),
 (u'N', 49718053, u'highway', u'crossing'),
 (u'N', 49721290, u'highway', u'crossing'),
 (u'N', 49723161, u'highway', u'stop'),
 (u'N', 49724802, u'highway', u'stop'),
 (u'N', 49724849, u'highway', u'stop'),
 (u'N', 49724953, u'highway', u'crossing'),
 (u'N', 49725736, u'highway', u'stop'),
 (u'N', 49726188, u'highway', u'stop'),
 (u'N', 49728780, u'highway', u'crossing'),
 (u'N', 49729003, u'highway', u'crossing'),
 (u'N', 49729884, u'highway', u'crossing'),
 (u'N', 49730917, u'highway', u'crossing'),
 (u'N', 49732057, u'highway', u'stop'),
 (u'N', 49732072, u'highway', u'stop'),
 (u'N', 49732073, u'highway', u'crossing'),
 (u'N', 49732609, u'highway', u'crossing

In [9]:
filter_tbtag(osm_db_dc, debug=True, in_tags=(('highway', ('stop','traffic_signals','crossing')),('traffic_signals', ['signal','crossing'])), ot='N')  # for comparision
filter_tbtag(osm_db_dc, debug=True, in_tags=(('highway', ('stop','traffic_signals','crossing')),('traffic_signals', ['signal','crossing'])), ex_tags=(('highway',['stop','crossing']),('traffic_signals','signal')), ot='N')  # ot+in_tags+ex_tags(multiple keys and multiple values)
filter_tbtag(osm_db_dc, debug=True, in_tags=(('highway',['stop','crossing']),('traffic_signals','signal')), ot='N')  # for comparision

filter table tag SELECT * FROM tag WHERE (ot='N') AND ((key='highway' and (value='stop' or value='traffic_signals' or value='crossing')) or (key='traffic_signals' and (value='signal' or value='crossing')))
# rows 4395
ot: [u'N']
# keys 2
keys: [u'highway' u'traffic_signals']
key = highway value: [u'stop' u'crossing' u'traffic_signals']
key = traffic_signals value: [u'signal' u'crossing']

filter table tag SELECT * FROM tag WHERE (ot='N') AND ((key='highway' and (value='stop' or value='traffic_signals' or value='crossing')) or (key='traffic_signals' and (value='signal' or value='crossing'))) AND (not((key='highway' and (value='stop' or value='crossing')) or (key='traffic_signals' and value='signal')))
# rows 1456
ot: [u'N']
# keys 2
keys: [u'highway' u'traffic_signals']
key = highway value: [u'traffic_signals']
key = traffic_signals value: [u'crossing']

filter table tag SELECT * FROM tag WHERE (ot='N') AND ((key='highway' and (value='stop' or value='crossing')) or (key='traffic_signals

[(u'N', 49715946, u'highway', u'stop'),
 (u'N', 49715951, u'highway', u'stop'),
 (u'N', 49715974, u'highway', u'stop'),
 (u'N', 49716326, u'highway', u'crossing'),
 (u'N', 49716545, u'traffic_signals', u'signal'),
 (u'N', 49716602, u'traffic_signals', u'signal'),
 (u'N', 49716610, u'traffic_signals', u'signal'),
 (u'N', 49716664, u'highway', u'crossing'),
 (u'N', 49716879, u'highway', u'stop'),
 (u'N', 49716928, u'highway', u'stop'),
 (u'N', 49716962, u'highway', u'stop'),
 (u'N', 49718053, u'highway', u'crossing'),
 (u'N', 49718775, u'traffic_signals', u'signal'),
 (u'N', 49718816, u'traffic_signals', u'signal'),
 (u'N', 49721290, u'highway', u'crossing'),
 (u'N', 49723161, u'highway', u'stop'),
 (u'N', 49723816, u'traffic_signals', u'signal'),
 (u'N', 49724285, u'traffic_signals', u'signal'),
 (u'N', 49724802, u'highway', u'stop'),
 (u'N', 49724849, u'highway', u'stop'),
 (u'N', 49724953, u'highway', u'crossing'),
 (u'N', 49725736, u'highway', u'stop'),
 (u'N', 49726188, u'highway', 

# Test cases for filter_tbtag distinct_otoid=True

In [22]:
filter_tbtag(osm_db_dc, in_tags=(('highway', ('stop','traffic_signals','crossing')),('traffic_signals', 'signal')), ot='N', debug=True, distinct_otoid=True)  # ot + in_tags(multiple keys, multiple values for one key)
filter_tbtag(osm_db_dc, debug=True, in_tags=(('highway', ('stop','traffic_signals','crossing')),('traffic_signals', 'signal')), ex_tags=(('highway','stop'),), ot='N', distinct_otoid=True)  # ot + in_tags + ex_tags(some_k=some_v)
filter_tbtag(osm_db_dc, debug=True, in_tags=(('highway','stop'),), ot='N', distinct_otoid=True)  # for comparision of whether ex_tags(some_k=some_v) works as expected


filter table tag: SELECT distinct ot, oid FROM tag WHERE (ot='N') AND ((key='highway' and (value='stop' or value='traffic_signals' or value='crossing')) or (key='traffic_signals' and value='signal'))
# rows 4217
ot: [u'N']

filter table tag: SELECT distinct ot, oid FROM tag WHERE (ot='N') AND ((key='highway' and (value='stop' or value='traffic_signals' or value='crossing')) or (key='traffic_signals' and value='signal')) AND (not((key='highway' and value='stop')))
# rows 3813
ot: [u'N']

filter table tag: SELECT distinct ot, oid FROM tag WHERE (ot='N') AND ((key='highway' and value='stop'))
# rows 404
ot: [u'N']



[(u'N', 49715946),
 (u'N', 49715951),
 (u'N', 49715974),
 (u'N', 49716879),
 (u'N', 49716928),
 (u'N', 49716962),
 (u'N', 49723161),
 (u'N', 49724802),
 (u'N', 49724849),
 (u'N', 49725736),
 (u'N', 49726188),
 (u'N', 49732057),
 (u'N', 49732072),
 (u'N', 49734010),
 (u'N', 49734413),
 (u'N', 49734802),
 (u'N', 49734873),
 (u'N', 49737931),
 (u'N', 49739319),
 (u'N', 49739803),
 (u'N', 49739997),
 (u'N', 49740100),
 (u'N', 49740442),
 (u'N', 49740477),
 (u'N', 49740585),
 (u'N', 49741041),
 (u'N', 49741043),
 (u'N', 49741052),
 (u'N', 49741058),
 (u'N', 49741076),
 (u'N', 49743717),
 (u'N', 49743760),
 (u'N', 49743812),
 (u'N', 49744292),
 (u'N', 49744348),
 (u'N', 49744352),
 (u'N', 49744361),
 (u'N', 49744382),
 (u'N', 49744407),
 (u'N', 49745354),
 (u'N', 49745407),
 (u'N', 49745415),
 (u'N', 49745427),
 (u'N', 49746439),
 (u'N', 49746516),
 (u'N', 49746854),
 (u'N', 49747084),
 (u'N', 49747094),
 (u'N', 49748172),
 (u'N', 49748231),
 (u'N', 49748901),
 (u'N', 49750468),
 (u'N', 4975

# func select_bk_facs

In [11]:
import qgrid, pandas as pd
rows = select_bk_facs(osm_db_dc,debug=True)
df = pd.DataFrame(rows)
qgrid.show_grid(df)

filter table tag SELECT * FROM tag WHERE ((key='highway' and (value='path' or value='pedestrian' or value='footway' or value='cycleway' or value='crossing' or value='track')) or (key='cycleway') or (key='cycleway:left') or (key='cycleway:right') or (key='cycleway:both') or (key='oneway:bicycle') or (key='bicycle') or (key='bicycle:lanes') or (key='bicycle:backward') or (key='amenity' and (value='bicycle_parking' or value='bicycle_rental')) or (key='foot') or (key='sidewalk') or (key='segregated'))
# rows 12933
ot: [u'N' u'W' u'R']
# keys 10



In [38]:
%load_ext sql

In [39]:
%sql sqlite:///data/osm_db_dc.sqlite3

u'Connected: None@data/osm_db_dc.sqlite3'

In [49]:
%%sql
SELECT ot,oid 
FROM geometry 
WHERE (ot,oid) IN (
    ('N', 30066952),
    ('N', 30066952)
)

(sqlite3.OperationalError) row value misused [SQL: u"SELECT ot,oid \nFROM geometry \nWHERE (ot,oid) IN (\n    ('N', 30066952),\n    ('N', 49716326)\n)"]


In [57]:
%%sql
SELECT * FROM geometry NATURAL JOIN (
  SELECT 'N' as ot, 30066952 as oid
UNION ALL
  SELECT 'N', 30066952
UNION ALL
  SELECT 'W', 97167308
);

(sqlite3.OperationalError) Could not decode to UTF-8 column 'wkb' with text '' [SQL: u"SELECT * FROM geometry NATURAL JOIN (\n  SELECT 'N' as ot, 30066952 as oid\nUNION ALL\n  SELECT 'N', 30066952\nUNION ALL\n  SELECT 'W', 97167308\n);"]


In [136]:
result = %sql SELECT DISTINCT * FROM tag WHERE (ot='N') AND ((key='traffic_signals'))
df = result.DataFrame()
df.sort('oid').head(10)

Done.


,ot,oid,key,value
0,N,49716545,traffic_signals,signal
1,N,49716602,traffic_signals,signal
2,N,49716610,traffic_signals,signal
3,N,49718775,traffic_signals,signal
4,N,49718816,traffic_signals,signal
5,N,49723816,traffic_signals,signal
6,N,49724285,traffic_signals,signal
7,N,49730831,traffic_signals,signal
8,N,49731188,traffic_signals,signal
9,N,49731194,traffic_signals,signal


In [101]:
result = %sql SELECT DISTINCT * FROM tag WHERE (ot='N') AND ((key='highway' and (value='stop' or value='traffic_signals')) or (key='traffic_signals' and value='signal'))
df = result.DataFrame()
df.sort('oid').head(10)

Done.


,ot,oid,key,value
0,N,49715946,highway,stop
1,N,49715951,highway,stop
2,N,49715974,highway,stop
3,N,49716545,highway,traffic_signals
4,N,49716545,traffic_signals,signal
5,N,49716602,highway,traffic_signals
6,N,49716602,traffic_signals,signal
7,N,49716610,highway,traffic_signals
8,N,49716610,traffic_signals,signal
9,N,49716662,highway,traffic_signals


In [137]:
print pd.unique(df.ot)
print pd.unique(df.key)
print pd.unique(df[df.key=='highway'].value)
print pd.unique(df[df.key=='traffic_signals'].value)

[u'N']
[u'traffic_signals']
[]
[u'signal' u'crossing']


In [94]:
result = %sql SELECT DISTINCT * FROM tag WHERE ((key='highway'))

df = result.DataFrame()

df.head(10)

Done.


,ot,oid,key,value
0,N,49715946,highway,stop
1,N,49715951,highway,stop
2,N,49715974,highway,stop
3,N,49716326,highway,crossing
4,N,49716545,highway,traffic_signals
5,N,49716602,highway,traffic_signals
6,N,49716610,highway,traffic_signals
7,N,49716662,highway,traffic_signals
8,N,49716664,highway,crossing
9,N,49716680,highway,traffic_signals


In [81]:
result = %sql select * from tag limit 10000

df = result.DataFrame()

df.head(10)

Done.


,ot,oid,key,value
0,N,30066952,created_by,JOSM
1,N,30066953,created_by,JOSM
2,N,30066954,created_by,JOSM
3,N,30066955,created_by,JOSM
4,N,49715946,highway,stop
5,N,49715951,highway,stop
6,N,49715974,highway,stop
7,N,49716326,highway,crossing
8,N,49716545,traffic_signals,signal
9,N,49716545,highway,traffic_signals
